### 1. Set-up

#### 1.1 Imports

In [143]:
# Imports
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import math
import time
import numpy as np
import sys
import argparse
import os
import matplotlib.pyplot as plt
import pandas as pd
import math

#### 1.2 CUDA

In [144]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### 2. Data Pre-Processing

#### 2.1 Helper Functions

In [145]:
# Encoding and decoding
def decode(vocab,corpus):
    
    text = ''
    for i in range(len(corpus)):
        wID = corpus[i]
        text = text + vocab[wID] + ' '
    return(text)

def encode(words,text):
    corpus = []
    tokens = text.split(' ')
    for t in tokens:
        try:
            wID = words[t][0]
        except:
            wID = words['<unk>'][0]
        corpus.append(wID)
    return(corpus)

def read_encode(file_name,vocab,words,corpus,threshold):
    
    wID = len(vocab)
    
    if threshold > -1:
        with open(file_name,'rt', encoding='utf8') as f:
            for line in f:
                line = line.replace('\n','')
                tokens = line.split(' ')
                for t in tokens:
                    try:
                        elem = words[t]
                    except:
                        elem = [wID,0]
                        vocab.append(t)
                        wID = wID + 1
                    elem[1] = elem[1] + 1
                    words[t] = elem

        temp = words
        words = {}
        vocab = []
        wID = 0
        words['<unk>'] = [wID,100]
        vocab.append('<unk>')
        for t in temp:
            if temp[t][1] >= threshold:
                vocab.append(t)
                wID = wID + 1
                words[t] = [wID,temp[t][1]]
            
                    
    with open(file_name,'rt', encoding='utf8') as f:
        for line in f:
            line = line.replace('\n','')
            tokens = line.split(' ')
            for t in tokens:
                try:
                    wID = words[t][0]
                except:
                    wID = words['<unk>'][0]
                corpus.append(wID)
                
    return [vocab,words,corpus]

def plot_data(x, y, xlabel, ylabel, title, color):
    plt.plot(x, y, color)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.show()

#### 2.2 Create ngrams

In [146]:
def split_bios(data, words):
  fake_val, real_val = words["[FAKE]"][0], words["[REAL]"][0]
  fake_list = (data == fake_val)
  real_list = (data == real_val)
  target_indices = (fake_list + real_list).nonzero()
  bios_list = []
  start_i = 0

  for end_i in target_indices:
    bio = data[start_i:end_i+1]
    bios_list.append(bio)

    start_i = end_i + 1

  return bios_list

def clean_bios(bios, words):
  for bio_i in range(len(bios)):
    # Remove new lines
    new_line_idx = words[''][0]
    comma_idx = words[','][0]
    period_idx = words['.'][0]
    colon_idx = words[':'][0]
    semicolon_idx = words[';'][0]

    punctuation_indices = [new_line_idx, comma_idx, period_idx, colon_idx, semicolon_idx]

    bios[bio_i] = list(filter(lambda x: x not in punctuation_indices, bios[bio_i]))

  return bios

def create_ngrams(bios, words, ngram_size):
  ngram_list = []

  for bio_i, bio in enumerate(bios):
    bio_len = len(bio)
    i = 0
    
    while i < bio_len - ngram_size:
      start, stop = i, i+ngram_size
      context = torch.tensor(bio[start:stop]).to(device)
      label = torch.tensor(bio[stop]).to(device)

      ngram = [context, label]
      ngram_list.append(ngram)

      i += 1

  return ngram_list


#### 2.3 Data Loader

In [147]:
class NgramDataset(Dataset):
    def __init__(self, ngrams_data):
        self.ngrams_data_df = pd.DataFrame(ngrams_data, columns=("context", "label"))
        self.context = self.ngrams_data_df["context"]
        self.label = self.ngrams_data_df["label"]

    def __len__(self):
        return len(self.label)

    def __getitem__(self, idx):
        return self.context[idx], self.label[idx]

### 3. Model

In [148]:
# FeedForward Model
class FFNN(nn.Module):
    def __init__(self, vocab, words, d_model, d_hidden, dropout, ngram_size):
        super().__init__() 
    
        # Class parameters
        self.vocab = vocab
        self.words = words
        self.vocab_size = len(self.vocab)
        self.d_model = d_model
        self.d_hidden = d_hidden
        self.ngram_size = ngram_size
        
        # Dropout
        self.dropout = nn.Dropout(p=dropout)

        # Embedding Layer
        self.input_embedding = nn.Embedding(self.vocab_size, self.d_model)

        # Linear Layers
        self.fc1 = nn.Linear(ngram_size * d_model, self.d_model)
        self.output_embedding = nn.Linear(self.d_model, self.vocab_size)

        # Nonlinear Layer
        self.activation = nn.ReLU()

        # Setting weights
        self.init_weights()
                
    # Initialize weights for foward layer
    def init_weights(self):
        weight_range = 0.1
        
        self.input_embedding.weight.data.uniform_(-weight_range, weight_range)
        self.fc1.weight.data.uniform_(-weight_range, weight_range)
        self.fc1.bias.data.zero_()

    # Forward
    def forward(self, src):
        # Embeddings are fed into the forward layer
        embeds = self.input_embedding(src).view(-1, self.d_model * self.ngram_size)
        x = self.dropout(self.activation(self.fc1(embeds)))
        x = self.output_embedding(x).view(-1, self.vocab_size)
        # x = F.log_softmax(x, dim=1)
        return x

### 4. Train

#### 4.1 Metrics

#### 4.2 Train One Epoch

In [149]:
def train_one_epoch(model, optimizer, criterion, train_dataloader):
  running_acc, running_loss = 0, 0
  num_batches = len(train_dataloader)
  
  for batch_idx, (train_contexts, train_labels) in enumerate(train_dataloader):
      model.zero_grad()

      logits = model(train_contexts)
      preds = torch.argmax(logits, dim=1)
      loss = criterion(logits, train_labels)
      acc = accuracy_score(train_labels, preds)

      running_loss += loss.item()
      running_acc += acc

      if batch_idx % 10000 == 0: 
        print(f"Batch: {batch_idx}")
        print(f"Loss: {running_loss}")
        print(f"Accuracy: {running_acc}\n")

      loss.backward()
      optimizer.step()

  av_acc = running_acc / num_batches
  av_loss = running_loss / num_batches

  return av_acc, av_loss


#### 4.3 Train Multiple Epochs

In [149]:
def train(model, optimizer, criterion, train_dataloader, epochs):
  train_accuracies = []
  train_losses = []
  epochs = list(range(epochs))

  for i in epochs:
    print(f"Epoch: {i+1} /////////////////////////////////////")
    av_acc, av_loss = train_one_epoch(model, optimizer, criterion, train_dataloader)

    print(f"Accuracy: {av_acc}")
    print(f"Loss: {av_loss}\n")

    train_accuracies.append(av_acc)
    train_losses.append(av_loss)

  # Graph Accuracies and Loss
  plot_data(x=epochs, y=train_accuracies, xlabel="Epochs", yLabel="Accuracy", title="Accuracy", color='r')
  plot_data(x=epochs, y=train_losses, xlabel="Epochs", yLabel="Loss", title="Loss", color='r')

### 5. Running Code


#### Loading Parameters

In [150]:
class Params:
    def __init__(self, **kwargs):
        for key, value in kwargs.items():
            setattr(self, key, value)
            
model_map = {0: 'FFNN', 1: 'LSTM', 2: 'FFNN_CLASSIFY', 3: 'LSTM_CLASSIFY'}
train_map = {0: 'data/real.train.tok', 1: 'data/fake.train.tok', 2: 'data/mix.train.tok'}
valid_map = {0: 'data/real.valid.tok', 1: 'data/fake.valid.tok', 2: 'data/mix.valid.tok'}
test_map = {0: 'data/real.test.tok', 1: 'data/fake.test.tok', 2: 'data/mix.test.tok', 3: 'data/blind.test.tok'}

model_type = model_map[0]

# Types of data
train_type = train_map[2]
valid_type = valid_map[2]
test_type = test_map[2]

args = {
    "d_model": 4,
    "d_hidden": 4,
    "n_layers": 3,
    "batch_size": 64,
    "seq_len": 30,
    "printevery": 5000,
    "window": 3,
    "epochs": 1,
    "lr": 0.0001,
    "dropout": 0.35,
    "clip": 2.0,
    "model": model_type,
    "savename": model_type.lower(),
    "loadname": model_type.lower(),
    "trainname": train_type,
    "validname": valid_type,
    "testname": test_type
}

#### Main

In [151]:
# Main Function
def main(args): 
    torch.manual_seed(0)
    
    # params
    params = Params(**args)
    train_name = params.trainname
    valid_name = params.validname
    test_name = params.testname
    model_type = params.model
    d_model = params.d_model
    d_hidden = params.d_hidden
    dropout = params.dropout
    epochs = params.epochs
    window = params.window
    batch_size = params.batch_size
    lr = params.lr

    # Extract vocab and words
    [train_vocab,train_words,train] = read_encode(train_name,[],{},[],3)
    train_data = torch.tensor(train)

    [valid_vocab,valid_words,valid] = read_encode(valid_name,[],{},[],3)
    valid_data = torch.tensor(valid)

    [test_vocab,test_words,test] = read_encode(test_name,[],{},[],3)
    test_data = torch.tensor(test)

    # Process Data
    train_bios = split_bios(train_data, train_words)
    train_bios = clean_bios(train_bios, train_words)
    train_ngrams_data = create_ngrams(train_bios, train_words, window)

    train_ngram_dataset = NgramDataset(train_ngrams_data)
    train_ngram_dataloader = DataLoader(train_ngram_dataset, batch_size=batch_size, shuffle=True)
    
    if model_type == 'FFNN':

      ngram_model = FFNN(train_vocab, train_words, d_model, d_hidden, dropout, window)
      optimizer = torch.optim.Adam(ngram_model.parameters(), lr=lr)
      criterion = nn.CrossEntropyLoss()

      train(ngram_model, optimizer, criterion, train_ngram_dataloader, epochs)

    if model_type == 'LSTM':
        pass
    if model_type == 'FFNN_CLASSIFY':
        pass
    if model_type == 'LSTM_CLASSIFY':
        pass


In [152]:
main(args)

<ipython-input-146-daad7673beab>:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(bio[stop]).to(device)


Streaming output truncated to the last 5000 lines.
Batch: 6230
Batch: 6231
Batch: 6232
Batch: 6233
Batch: 6234
Batch: 6235
Batch: 6236
Batch: 6237
Batch: 6238
Batch: 6239
Batch: 6240
Batch: 6241
Batch: 6242
Batch: 6243
Batch: 6244
Batch: 6245
Batch: 6246
Batch: 6247
Batch: 6248
Batch: 6249
Batch: 6250
Batch: 6251
Batch: 6252
Batch: 6253
Batch: 6254
Batch: 6255
Batch: 6256
Batch: 6257
Batch: 6258
Batch: 6259
Batch: 6260
Batch: 6261
Batch: 6262
Batch: 6263
Batch: 6264
Batch: 6265
Batch: 6266
Batch: 6267
Batch: 6268
Batch: 6269
Batch: 6270
Batch: 6271
Batch: 6272
Batch: 6273
Batch: 6274
Batch: 6275
Batch: 6276
Batch: 6277
Batch: 6278
Batch: 6279
Batch: 6280
Batch: 6281
Batch: 6282
Batch: 6283
Batch: 6284
Batch: 6285
Batch: 6286
Batch: 6287
Batch: 6288
Batch: 6289
Batch: 6290
Batch: 6291
Batch: 6292
Batch: 6293
Batch: 6294
Batch: 6295
Batch: 6296
Batch: 6297
Batch: 6298
Batch: 6299
Batch: 6300
Batch: 6301
Batch: 6302
Batch: 6303
Batch: 6304
Batch: 6305
Batch: 6306
Batch: 6307
Batch: 6308
B

KeyboardInterrupt: ignored